# Using Callbacks

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

In [2]:
# loading and splitting the dataset into train, valid and test set
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

In [4]:
model = keras.models.Sequential([
                                 keras.layers.Dense(30, activation="relu", input_shape=[8]),
                                 keras.layers.Dense(30, activation="relu"),
                                 keras.layers.Dense(1)
])

The fit() method accepts a callbacks argument that lets you specify a list of objects that Keras will call at the start and end of training, at the start and end of each epoch, and even before and after processing each batch. For example, the ModelCheckpoint callback saves checkpoints of your model at regular intervals during training, by default at the end of each epoch: 

If training lasts several hourse, you should not only save your model at the end of training, but also save checkpoints at regular intervals during training, to avoid losing everything if computer crashes

Moreover, if you use a validation set during training, you can set save_best_only=True when creating the ModelCheckpoint. In this case, it will only save your model when its performance on the validation set is the best so far. This way, you do not need to worry about training for too long and overfitting the training set: simply restore the last model saved after training, and this will be the best model on the validation set. 

In [7]:
model.compile(loss="mse", optimizer=keras.optimizers.SGD(lr=1e-3))

checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

history = model.fit(X_train, y_train, validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb], epochs=10)

model = keras.models.load_model("my_keras_model.h5") # rollback to best model

Epoch 1/10
363/363 [==============================] - 1s 1ms/step - loss: 0.6635 - val_loss: 0.7135
Epoch 2/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5899 - val_loss: 0.6581
Epoch 3/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5495 - val_loss: 0.6276
Epoch 4/10
363/363 [==============================] - 0s 1ms/step - loss: 0.5196 - val_loss: 0.6031
Epoch 5/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4963 - val_loss: 0.5841
Epoch 6/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4780 - val_loss: 0.5670
Epoch 7/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4633 - val_loss: 0.5549
Epoch 8/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4512 - val_loss: 0.5486
Epoch 9/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4412 - val_loss: 0.5379
Epoch 10/10
363/363 [==============================] - 0s 1ms/step - loss: 0.4326 - val_loss: 0.5350

In [8]:
mse_test = model.evaluate(X_test, y_test)

162/162 [==============================] - 0s 796us/step - loss: 0.5236


### Early Stopping and callbacks

To implement Early stopping simply use the EarlyStopping callback. It will interrupt training when it measures no progress on the validation set for a number of epochs (defined by the patience argument), and it will optionally roll back to the best model. You can combine both callbacks to save checkpoints of your model (in case your computer crashes) and interrupt training early when there is no more progress (to avoid wasting time and resources):


In [9]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid),
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4254 - val_loss: 0.5279
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4191 - val_loss: 0.5213
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4135 - val_loss: 0.5168
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4083 - val_loss: 0.5160
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 0.4038 - val_loss: 0.5099
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3998 - val_loss: 0.5106
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3962 - val_loss: 0.5047
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3928 - val_loss: 0.5032
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3894 - val_loss: 0.5052
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 0.3866 - val_lo

The number of epochs can be set to a large value since training will stop automati‐ cally when there is no more progress. In this case, there is no need to restore the best model saved because the EarlyStopping callback will keep track of the best weights and restore them for you at the end of training.

### Writing custom callbacks

In [12]:
class PrintValTrainRatioCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        print("\nval/train: {:.2f}".format(logs["val_loss"] / logs["loss"]))

In [13]:
val_train_ratio_cb = PrintValTrainRatioCallback()
history = model.fit(X_train, y_train, epochs=1,
                    validation_data=(X_valid, y_valid),
                    callbacks=[val_train_ratio_cb])

360/363 [============================>.] - ETA: 0s - loss: 0.3516
val/train: 1.38
363/363 [==============================] - 0s 1ms/step - loss: 0.3516 - val_loss: 0.4840


#### More on callbacks: 
- [Keras docs](https://keras.io/api/callbacks/)
- [TensorFlow docs](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback)